# coding:utf-8
验证码识别




In [1]:
# coding:utf-8
from captcha.image import ImageCaptcha
import random
from PIL import Image
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt

number = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']
ALPHABET = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U',
            'V', 'W', 'X', 'Y', 'Z']
SAVE_PATH = "D:\\Programming\\TF2.0 Verification Code" # 保存训练模型的路径
CHAR_SET = number + alphabet + ALPHABET
CHAR_SET_LEN = len(CHAR_SET)


#验证码图片的尺寸

IMAGE_HEIGHT = 60
IMAGE_WIDTH = 160

# 从文件夹里获取验证码名字及图片并转化为矩阵形式
def gen_captcha_text_and_image():
    all_image = os.listdir('D:/VCR_test_data/')
    random_file = random.randint(0, 1000)
    base = os.path.basename('D:/VCR_test_data/' + all_image[random_file])
    name = os.path.splitext(base)[0]
    img = Image.open('D:/VCR_test_data/' + all_image[random_file])
    image = np.array(img)
    return name, image


text, image = gen_captcha_text_and_image()
print(text)
MAX_CAPTCHA = len(text)
print('CHAR_SET_LEN=', CHAR_SET_LEN, ' MAX_CAPTCHA=', MAX_CAPTCHA)

# 转化为灰度图
def convert2gray(img):
    if len(img.shape) > 2:
        gray = np.mean(img, -1)
        return gray
    else:
        return img

#将验证码标签转化为向量
def text2vec(text):
    vector = np.zeros([MAX_CAPTCHA, CHAR_SET_LEN])
    for i, c in enumerate(text):
        idx = CHAR_SET.index(c)
        vector[i][idx] = 1.0
    return vector

# 将向量转化成验证码标签
def vec2text(vec):
    text = []
    for i, c in enumerate(vec):
        text.append(CHAR_SET[c])
    return "".join(text)

#批量获取验证码图片
def get_next_batch(batch_size=128):
    batch_x = np.zeros([batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, 1])
    
    
    batch_y = np.zeros([batch_size, MAX_CAPTCHA, CHAR_SET_LEN])

    
    
    def wrap_gen_captcha_text_and_image():
        while True:
            text, image = gen_captcha_text_and_image()
            if image.shape == (60, 160, 3):
                return text, image

    for i in range(batch_size):
        text, image = wrap_gen_captcha_text_and_image()
        image = tf.reshape(convert2gray(image), (IMAGE_HEIGHT, IMAGE_WIDTH, 1))
        batch_x[i, :] = image
        batch_y[i, :] = text2vec(text)

    return batch_x, batch_y

# CNN模型，3层卷积层，1层全连接
def crack_captcha_cnn():
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Conv2D(32, (3, 3)))
    model.add(tf.keras.layers.PReLU())
    model.add(tf.keras.layers.MaxPool2D((2, 2), strides=2))

    model.add(tf.keras.layers.Conv2D(64, (5, 5)))
    model.add(tf.keras.layers.PReLU())
    model.add(tf.keras.layers.MaxPool2D((2, 2), strides=2))

    model.add(tf.keras.layers.Conv2D(128, (5, 5)))
    model.add(tf.keras.layers.PReLU())
    model.add(tf.keras.layers.MaxPool2D((2, 2), strides=2))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(MAX_CAPTCHA * CHAR_SET_LEN))
    model.add(tf.keras.layers.Reshape([MAX_CAPTCHA, CHAR_SET_LEN]))

    model.add(tf.keras.layers.Softmax())

    return model

# 模型训练函数
def train():
    try:
        model = tf.keras.models.load_model(SAVE_PATH + 'model')
    except Exception as e:
        print('#######Exception', e)
        model = crack_captcha_cnn()

    model.compile(optimizer='Adam',
                  metrics=['accuracy'],
                  loss='categorical_crossentropy')

    for times in range(1000):
        batch_x, batch_y = get_next_batch(100)
        print('times=', times, ' batch_x.shape=', batch_x.shape, ' batch_y.shape=', batch_y.shape)
        model.fit(batch_x, batch_y, epochs=4)
        # 训练1000次时保持模型
        if times == 999:
            print("save model at times=", times)
            model.save(SAVE_PATH + 'model')

# 验证码预测函数，应用测试集
def predict():
    model = tf.keras.models.load_model(SAVE_PATH + 'model')
    success = 0
    count = 1000
    for _ in range(count):
        data_x, data_y = get_next_batch(1)
        prediction_value = model.predict(data_x)
        data_y = vec2text(np.argmax(data_y, axis=2)[0])
        prediction_value = vec2text(np.argmax(prediction_value, axis=2)[0])

        if data_y.upper() == prediction_value.upper():
            print("y预测=", prediction_value, "y实际=", data_y, "预测成功。")
            success += 1
        else:
            print("y预测=", prediction_value, "y实际=", data_y, "预测失败。")
        print("预测", count, "次", "成功率=", success / count)

    pass


if __name__ == "__main__":
    train()
    

D:\Anaconda\envs\tensorflow\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
D:\Anaconda\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


BHbS
CHAR_SET_LEN= 62  MAX_CAPTCHA= 4
#######Exception SavedModel file does not exist at: D:\Programming\TF2.0 Verification Codemodel/{saved_model.pbtxt|saved_model.pb}
times= 0  batch_x.shape= (100, 60, 160, 1)  batch_y.shape= (100, 4, 62)
Epoch 1/4
4/4 [==============================] - 1s 250ms/step - loss: 49.7739 - accuracy: 0.0350
Epoch 2/4
4/4 [==============================] - 1s 256ms/step - loss: 8.8048 - accuracy: 0.0175
Epoch 3/4
4/4 [==============================] - 1s 256ms/step - loss: 3.8595 - accuracy: 0.0550
Epoch 4/4
4/4 [==============================] - 1s 259ms/step - loss: 3.7538 - accuracy: 0.0475
times= 1  batch_x.shape= (100, 60, 160, 1)  batch_y.shape= (100, 4, 62)
Epoch 1/4
4/4 [==============================] - 1s 257ms/step - loss: 4.1513 - accuracy: 0.0200
Epoch 2/4
4/4 [==============================] - 1s 262ms/step - loss: 3.8847 - accuracy: 0.0225
Epoch 3/4
4/4 [==============================] - 1s 265ms/step - loss: 3.7356 - accuracy: 0.0500
Epoch 4

4/4 [==============================] - 1s 284ms/step - loss: 4.0398 - accuracy: 0.0275
Epoch 2/4
4/4 [==============================] - 1s 290ms/step - loss: 3.8284 - accuracy: 0.0375
Epoch 3/4
4/4 [==============================] - 1s 292ms/step - loss: 3.7614 - accuracy: 0.0450
Epoch 4/4
4/4 [==============================] - 1s 286ms/step - loss: 3.7179 - accuracy: 0.0575
times= 36  batch_x.shape= (100, 60, 160, 1)  batch_y.shape= (100, 4, 62)
Epoch 1/4
4/4 [==============================] - 1s 276ms/step - loss: 3.9685 - accuracy: 0.0275
Epoch 2/4
4/4 [==============================] - 1s 277ms/step - loss: 3.8010 - accuracy: 0.0300
Epoch 3/4
4/4 [==============================] - 1s 282ms/step - loss: 3.7093 - accuracy: 0.0450
Epoch 4/4
4/4 [==============================] - 1s 285ms/step - loss: 3.6639 - accuracy: 0.0550
times= 37  batch_x.shape= (100, 60, 160, 1)  batch_y.shape= (100, 4, 62)
Epoch 1/4
4/4 [==============================] - 1s 273ms/step - loss: 4.0949 - accuracy

4/4 [==============================] - 1s 282ms/step - loss: 4.3786 - accuracy: 0.0425
Epoch 4/4
4/4 [==============================] - 1s 278ms/step - loss: 4.2022 - accuracy: 0.0300
times= 71  batch_x.shape= (100, 60, 160, 1)  batch_y.shape= (100, 4, 62)
Epoch 1/4
4/4 [==============================] - 1s 277ms/step - loss: 4.0521 - accuracy: 0.0200
Epoch 2/4
4/4 [==============================] - 1s 283ms/step - loss: 3.8142 - accuracy: 0.0475
Epoch 3/4
4/4 [==============================] - 1s 295ms/step - loss: 3.7667 - accuracy: 0.0600
Epoch 4/4
4/4 [==============================] - 1s 292ms/step - loss: 3.7452 - accuracy: 0.0275
times= 72  batch_x.shape= (100, 60, 160, 1)  batch_y.shape= (100, 4, 62)
Epoch 1/4
4/4 [==============================] - 1s 286ms/step - loss: 3.9757 - accuracy: 0.0150
Epoch 2/4
4/4 [==============================] - 1s 282ms/step - loss: 3.7292 - accuracy: 0.0600
Epoch 3/4
4/4 [==============================] - 1s 283ms/step - loss: 3.6724 - accuracy

4/4 [==============================] - 1s 279ms/step - loss: 3.8483 - accuracy: 0.0275
Epoch 3/4
4/4 [==============================] - 1s 281ms/step - loss: 3.8181 - accuracy: 0.0350
Epoch 4/4
4/4 [==============================] - 1s 280ms/step - loss: 3.7862 - accuracy: 0.0350
times= 89  batch_x.shape= (100, 60, 160, 1)  batch_y.shape= (100, 4, 62)
Epoch 1/4
4/4 [==============================] - 1s 281ms/step - loss: 3.8384 - accuracy: 0.0250
Epoch 2/4
4/4 [==============================] - 1s 284ms/step - loss: 3.8261 - accuracy: 0.0250
Epoch 3/4
4/4 [==============================] - 1s 281ms/step - loss: 3.8052 - accuracy: 0.0325
Epoch 4/4
4/4 [==============================] - 1s 279ms/step - loss: 3.7841 - accuracy: 0.0375
times= 90  batch_x.shape= (100, 60, 160, 1)  batch_y.shape= (100, 4, 62)
Epoch 1/4
4/4 [==============================] - 1s 281ms/step - loss: 3.8729 - accuracy: 0.0225
Epoch 2/4
4/4 [==============================] - 1s 285ms/step - loss: 3.8558 - accuracy

...

4/4 [==============================] - 1s 287ms/step - loss: 1.0930 - accuracy: 0.9000
Epoch 2/4
4/4 [==============================] - 1s 289ms/step - loss: 0.0360 - accuracy: 0.9850
Epoch 3/4
4/4 [==============================] - 1s 284ms/step - loss: 0.0081 - accuracy: 0.9925
Epoch 4/4
4/4 [==============================] - 1s 286ms/step - loss: 0.0025 - accuracy: 1.0000
times= 866  batch_x.shape= (100, 60, 160, 1)  batch_y.shape= (100, 4, 62)
Epoch 1/4
4/4 [==============================] - 1s 284ms/step - loss: 1.1399 - accuracy: 0.8975
Epoch 2/4
4/4 [==============================] - 1s 283ms/step - loss: 0.1650 - accuracy: 0.9700
Epoch 3/4
4/4 [==============================] - 1s 283ms/step - loss: 0.3441 - accuracy: 0.9500
Epoch 4/4
4/4 [==============================] - 1s 285ms/step - loss: 0.4000 - accuracy: 0.9700
times= 867  batch_x.shape= (100, 60, 160, 1)  batch_y.shape= (100, 4, 62)
Epoch 1/4
4/4 [==============================] - 1s 278ms/step - loss: 1.7328 - accura

4/4 [==============================] - 1s 279ms/step - loss: 4.2804e-04 - accuracy: 1.0000
Epoch 3/4
4/4 [==============================] - 1s 282ms/step - loss: 1.9972e-04 - accuracy: 1.0000
Epoch 4/4
4/4 [==============================] - 1s 286ms/step - loss: 1.2766e-04 - accuracy: 1.0000
times= 901  batch_x.shape= (100, 60, 160, 1)  batch_y.shape= (100, 4, 62)
Epoch 1/4
4/4 [==============================] - 1s 279ms/step - loss: 0.5204 - accuracy: 0.9350
Epoch 2/4
4/4 [==============================] - 1s 284ms/step - loss: 3.9757e-04 - accuracy: 1.0000
Epoch 3/4
4/4 [==============================] - 1s 289ms/step - loss: 0.0310 - accuracy: 0.9950
Epoch 4/4
4/4 [==============================] - 1s 293ms/step - loss: 8.8651e-05 - accuracy: 1.0000
times= 902  batch_x.shape= (100, 60, 160, 1)  batch_y.shape= (100, 4, 62)
Epoch 1/4
4/4 [==============================] - 1s 280ms/step - loss: 0.6463 - accuracy: 0.9500
Epoch 2/4
4/4 [==============================] - 1s 288ms/step - l

4/4 [==============================] - 1s 282ms/step - loss: 1.0602 - accuracy: 0.9250
Epoch 2/4
4/4 [==============================] - 1s 280ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 3/4
4/4 [==============================] - 1s 308ms/step - loss: 8.8097e-04 - accuracy: 1.0000
Epoch 4/4
4/4 [==============================] - 1s 289ms/step - loss: 0.0028 - accuracy: 0.9975
times= 919  batch_x.shape= (100, 60, 160, 1)  batch_y.shape= (100, 4, 62)
Epoch 1/4
4/4 [==============================] - 1s 279ms/step - loss: 0.9115 - accuracy: 0.9375
Epoch 2/4
4/4 [==============================] - 1s 279ms/step - loss: 0.2119 - accuracy: 0.9825
Epoch 3/4
4/4 [==============================] - 1s 281ms/step - loss: 0.1030 - accuracy: 0.9775
Epoch 4/4
4/4 [==============================] - 1s 293ms/step - loss: 0.0225 - accuracy: 0.9950
times= 920  batch_x.shape= (100, 60, 160, 1)  batch_y.shape= (100, 4, 62)
Epoch 1/4
4/4 [==============================] - 1s 290ms/step - loss: 0.3197 - ac

Epoch 4/4
4/4 [==============================] - 1s 282ms/step - loss: 0.0033 - accuracy: 0.9975
times= 936  batch_x.shape= (100, 60, 160, 1)  batch_y.shape= (100, 4, 62)
Epoch 1/4
4/4 [==============================] - 1s 277ms/step - loss: 0.8236 - accuracy: 0.9400
Epoch 2/4
4/4 [==============================] - 1s 284ms/step - loss: 0.0157 - accuracy: 0.9925
Epoch 3/4
4/4 [==============================] - 1s 285ms/step - loss: 0.0246 - accuracy: 0.9900
Epoch 4/4
4/4 [==============================] - 1s 288ms/step - loss: 0.0010 - accuracy: 1.0000
times= 937  batch_x.shape= (100, 60, 160, 1)  batch_y.shape= (100, 4, 62)
Epoch 1/4
4/4 [==============================] - 1s 281ms/step - loss: 0.7850 - accuracy: 0.9400
Epoch 2/4
4/4 [==============================] - 1s 283ms/step - loss: 0.0336 - accuracy: 0.9925
Epoch 3/4
4/4 [==============================] - 1s 280ms/step - loss: 0.0478 - accuracy: 0.9850
Epoch 4/4
4/4 [==============================] - 1s 284ms/step - loss: 0.007

4/4 [==============================] - 1s 276ms/step - loss: 1.4917 - accuracy: 0.8900
Epoch 2/4
4/4 [==============================] - 1s 283ms/step - loss: 0.0415 - accuracy: 0.9950
Epoch 3/4
4/4 [==============================] - 1s 283ms/step - loss: 0.0923 - accuracy: 0.9875
Epoch 4/4
4/4 [==============================] - 1s 287ms/step - loss: 0.0827 - accuracy: 0.9900
times= 972  batch_x.shape= (100, 60, 160, 1)  batch_y.shape= (100, 4, 62)
Epoch 1/4
4/4 [==============================] - 1s 274ms/step - loss: 1.8388 - accuracy: 0.9050
Epoch 2/4
4/4 [==============================] - 1s 285ms/step - loss: 0.0676 - accuracy: 0.9900
Epoch 3/4
4/4 [==============================] - 1s 283ms/step - loss: 0.0116 - accuracy: 0.9950
Epoch 4/4
4/4 [==============================] - 1s 287ms/step - loss: 0.0098 - accuracy: 0.9975
times= 973  batch_x.shape= (100, 60, 160, 1)  batch_y.shape= (100, 4, 62)
Epoch 1/4
4/4 [==============================] - 1s 276ms/step - loss: 0.7732 - accura

In [3]:
predict()

y预测= M8Z1 y实际= M8Z1 预测成功。
预测 1000 次 成功率= 0.001
y预测= XmWN y实际= X0WN 预测失败。
预测 1000 次 成功率= 0.001
y预测= FT4B y实际= FXJB 预测失败。
预测 1000 次 成功率= 0.001
y预测= DG4e y实际= DG4e 预测成功。
预测 1000 次 成功率= 0.002
y预测= X9mK y实际= X9mK 预测成功。
预测 1000 次 成功率= 0.003
y预测= B3Py y实际= B3Py 预测成功。
预测 1000 次 成功率= 0.004
y预测= kery y实际= kery 预测成功。
预测 1000 次 成功率= 0.005
y预测= P4NL y实际= 94NL 预测失败。
预测 1000 次 成功率= 0.005
y预测= hQKr y实际= hQ6r 预测失败。
预测 1000 次 成功率= 0.005
y预测= 1tX0 y实际= ytb0 预测失败。
预测 1000 次 成功率= 0.005
y预测= n3RY y实际= n3RY 预测成功。
预测 1000 次 成功率= 0.006
y预测= eULN y实际= eULN 预测成功。
预测 1000 次 成功率= 0.007
y预测= 1aP1 y实际= JaP1 预测失败。
预测 1000 次 成功率= 0.007
y预测= VmCT y实际= VmCT 预测成功。
预测 1000 次 成功率= 0.008
y预测= HE4f y实际= HE4f 预测成功。
预测 1000 次 成功率= 0.009
y预测= VCPG y实际= VCPG 预测成功。
预测 1000 次 成功率= 0.01
y预测= 8eNa y实际= 8eNa 预测成功。
预测 1000 次 成功率= 0.011
y预测= GnTN y实际= GPTN 预测失败。
预测 1000 次 成功率= 0.011
y预测= aYSE y实际= aYmE 预测失败。
预测 1000 次 成功率= 0.011
y预测= dS2W y实际= dS2W 预测成功。
预测 1000 次 成功率= 0.012
y预测= 5n2T y实际= 5n2T 预测成功。
预测 1000 次 成功率= 0.013
y预测= SUW0 y实际=

y预测= KJ5B y实际= KJ5B 预测成功。
预测 1000 次 成功率= 0.141
y预测= 00fZ y实际= 00fZ 预测成功。
预测 1000 次 成功率= 0.142
y预测= ymHY y实际= y7HY 预测失败。
预测 1000 次 成功率= 0.142
y预测= 2HDB y实际= 2HDB 预测成功。
预测 1000 次 成功率= 0.143
y预测= 3ZPn y实际= 3ZPn 预测成功。
预测 1000 次 成功率= 0.144
y预测= ZfF4 y实际= ZfF4 预测成功。
预测 1000 次 成功率= 0.145
y预测= ZCQT y实际= ZCQT 预测成功。
预测 1000 次 成功率= 0.146
y预测= FTV7 y实际= FTVM 预测失败。
预测 1000 次 成功率= 0.146
y预测= 3DWt y实际= 3DWt 预测成功。
预测 1000 次 成功率= 0.147
y预测= 1dDA y实际= 1dDA 预测成功。
预测 1000 次 成功率= 0.148
y预测= VPZA y实际= VyZA 预测失败。
预测 1000 次 成功率= 0.148
y预测= bDbT y实际= bDbT 预测成功。
预测 1000 次 成功率= 0.149
y预测= yVBN y实际= yVBN 预测成功。
预测 1000 次 成功率= 0.15
y预测= hXQZ y实际= hXQZ 预测成功。
预测 1000 次 成功率= 0.151
y预测= 3U4h y实际= 5URh 预测失败。
预测 1000 次 成功率= 0.151
y预测= Q1AB y实际= Q1AB 预测成功。
预测 1000 次 成功率= 0.152
y预测= mhyR y实际= mhyR 预测成功。
预测 1000 次 成功率= 0.153
y预测= 5SCD y实际= 5SCD 预测成功。
预测 1000 次 成功率= 0.154
y预测= 9DCC y实际= 9DCC 预测成功。
预测 1000 次 成功率= 0.155
y预测= 7Z5R y实际= 7Z5R 预测成功。
预测 1000 次 成功率= 0.156
y预测= MSAD y实际= MSAD 预测成功。
预测 1000 次 成功率= 0.157
y预测= 1hbm y实际=

y预测= dnaM y实际= dnaM 预测成功。
预测 1000 次 成功率= 0.278
y预测= ZmMB y实际= Z7MB 预测失败。
预测 1000 次 成功率= 0.278
y预测= mb7M y实际= mb7M 预测成功。
预测 1000 次 成功率= 0.279
y预测= A6A0 y实际= A6A0 预测成功。
预测 1000 次 成功率= 0.28
y预测= DQKP y实际= DQKP 预测成功。
预测 1000 次 成功率= 0.281
y预测= DyMZ y实际= DyMZ 预测成功。
预测 1000 次 成功率= 0.282
y预测= J966 y实际= J966 预测成功。
预测 1000 次 成功率= 0.283
y预测= AbnT y实际= AbnT 预测成功。
预测 1000 次 成功率= 0.284
y预测= 01GF y实际= 01GF 预测成功。
预测 1000 次 成功率= 0.285
y预测= mEAd y实际= mEAW 预测失败。
预测 1000 次 成功率= 0.285
y预测= LEGH y实际= LEGH 预测成功。
预测 1000 次 成功率= 0.286
y预测= HXD6 y实际= HXD6 预测成功。
预测 1000 次 成功率= 0.287
y预测= emAk y实际= emA5 预测失败。
预测 1000 次 成功率= 0.287
y预测= BDnV y实际= BDnV 预测成功。
预测 1000 次 成功率= 0.288
y预测= ZCVN y实际= ZCVL 预测失败。
预测 1000 次 成功率= 0.288
y预测= d65J y实际= d65J 预测成功。
预测 1000 次 成功率= 0.289
y预测= StYe y实际= StYe 预测成功。
预测 1000 次 成功率= 0.29
y预测= M8Z1 y实际= M8Z1 预测成功。
预测 1000 次 成功率= 0.291
y预测= mC9r y实际= yD9r 预测失败。
预测 1000 次 成功率= 0.291
y预测= GLLX y实际= yLLX 预测失败。
预测 1000 次 成功率= 0.291
y预测= JGVb y实际= JGVb 预测成功。
预测 1000 次 成功率= 0.292
y预测= QatM y实际= 

y预测= brkJ y实际= brkJ 预测成功。
预测 1000 次 成功率= 0.431
y预测= dmXt y实际= dmXt 预测成功。
预测 1000 次 成功率= 0.432
y预测= U4JT y实际= U4JT 预测成功。
预测 1000 次 成功率= 0.433
y预测= NSKZ y实际= NSKZ 预测成功。
预测 1000 次 成功率= 0.434
y预测= BRmJ y实际= BRmJ 预测成功。
预测 1000 次 成功率= 0.435
y预测= 4mn1 y实际= 4mn1 预测成功。
预测 1000 次 成功率= 0.436
y预测= m3fQ y实际= m3fQ 预测成功。
预测 1000 次 成功率= 0.437
y预测= Un4h y实际= Un4h 预测成功。
预测 1000 次 成功率= 0.438
y预测= trVa y实际= trVa 预测成功。
预测 1000 次 成功率= 0.439
y预测= A2tH y实际= A2tH 预测成功。
预测 1000 次 成功率= 0.44
y预测= WtB3 y实际= WtB3 预测成功。
预测 1000 次 成功率= 0.441
y预测= XkyY y实际= XkyY 预测成功。
预测 1000 次 成功率= 0.442
y预测= KBdL y实际= KBdL 预测成功。
预测 1000 次 成功率= 0.443
y预测= MATa y实际= MATa 预测成功。
预测 1000 次 成功率= 0.444
y预测= PWf1 y实际= PWfD 预测失败。
预测 1000 次 成功率= 0.444
y预测= SSrf y实际= WSrf 预测失败。
预测 1000 次 成功率= 0.444
y预测= DeBF y实际= 8e3A 预测失败。
预测 1000 次 成功率= 0.444
y预测= 7VEb y实际= 7VEb 预测成功。
预测 1000 次 成功率= 0.445
y预测= BAfA y实际= BAfA 预测成功。
预测 1000 次 成功率= 0.446
y预测= QtQU y实际= QJQU 预测失败。
预测 1000 次 成功率= 0.446
y预测= UEMH y实际= UEMH 预测成功。
预测 1000 次 成功率= 0.447
y预测= dA6b y实际=

y预测= 9CR1 y实际= 9CR1 预测成功。
预测 1000 次 成功率= 0.573
y预测= nP3X y实际= nP3X 预测成功。
预测 1000 次 成功率= 0.574
y预测= ehC2 y实际= ehC2 预测成功。
预测 1000 次 成功率= 0.575
y预测= 5YNd y实际= 5YNd 预测成功。
预测 1000 次 成功率= 0.576
y预测= 26RT y实际= 26RT 预测成功。
预测 1000 次 成功率= 0.577
y预测= 5dYC y实际= 5dYC 预测成功。
预测 1000 次 成功率= 0.578
y预测= eJnF y实际= eJnF 预测成功。
预测 1000 次 成功率= 0.579
y预测= 0PRP y实际= 0PRP 预测成功。
预测 1000 次 成功率= 0.58
y预测= 4bnX y实际= 4bnX 预测成功。
预测 1000 次 成功率= 0.581
y预测= F4NU y实际= F4NU 预测成功。
预测 1000 次 成功率= 0.582
y预测= BYX6 y实际= BYX6 预测成功。
预测 1000 次 成功率= 0.583
y预测= yBBD y实际= yBBD 预测成功。
预测 1000 次 成功率= 0.584
y预测= 56eJ y实际= y6eJ 预测失败。
预测 1000 次 成功率= 0.584
y预测= kEYC y实际= kEYC 预测成功。
预测 1000 次 成功率= 0.585
y预测= n29C y实际= n29C 预测成功。
预测 1000 次 成功率= 0.586
y预测= rPSr y实际= rPSr 预测成功。
预测 1000 次 成功率= 0.587
y预测= F6XJ y实际= F6XJ 预测成功。
预测 1000 次 成功率= 0.588
y预测= RCVQ y实际= YCVQ 预测失败。
预测 1000 次 成功率= 0.588
y预测= 74EG y实际= 74EU 预测失败。
预测 1000 次 成功率= 0.588
y预测= BnLG y实际= 6nLG 预测失败。
预测 1000 次 成功率= 0.588
y预测= fZ20 y实际= fZ20 预测成功。
预测 1000 次 成功率= 0.589
y预测= Vay4 y实际=

y预测= 5WG9 y实际= 5WG9 预测成功。
预测 1000 次 成功率= 0.717
y预测= yk6A y实际= yk6A 预测成功。
预测 1000 次 成功率= 0.718
y预测= S6Ed y实际= S6Ed 预测成功。
预测 1000 次 成功率= 0.719
y预测= Fb96 y实际= Fb96 预测成功。
预测 1000 次 成功率= 0.72
y预测= r30C y实际= r30C 预测成功。
预测 1000 次 成功率= 0.721
y预测= rmAN y实际= rmZ0 预测失败。
预测 1000 次 成功率= 0.721
y预测= a0KX y实际= a0bX 预测失败。
预测 1000 次 成功率= 0.721
y预测= ybK1 y实际= ybK1 预测成功。
预测 1000 次 成功率= 0.722
y预测= QtmE y实际= QtmE 预测成功。
预测 1000 次 成功率= 0.723
y预测= nT2M y实际= nT2M 预测成功。
预测 1000 次 成功率= 0.724
y预测= Vfra y实际= Vfra 预测成功。
预测 1000 次 成功率= 0.725
y预测= 06ST y实际= 06ST 预测成功。
预测 1000 次 成功率= 0.726
y预测= HQVN y实际= HQVN 预测成功。
预测 1000 次 成功率= 0.727
y预测= YUJn y实际= YUJn 预测成功。
预测 1000 次 成功率= 0.728
y预测= TZEX y实际= TZEG 预测失败。
预测 1000 次 成功率= 0.728
y预测= r9RP y实际= r9RP 预测成功。
预测 1000 次 成功率= 0.729
y预测= Pk8Y y实际= Pk8Y 预测成功。
预测 1000 次 成功率= 0.73
y预测= ECL1 y实际= ECL1 预测成功。
预测 1000 次 成功率= 0.731
y预测= NXLL y实际= NXLL 预测成功。
预测 1000 次 成功率= 0.732
y预测= yAKM y实际= yAKM 预测成功。
预测 1000 次 成功率= 0.733
y预测= rMmA y实际= rMmy 预测失败。
预测 1000 次 成功率= 0.733
y预测= GaQR y实际= 

y预测= 2HaZ y实际= 2HaZ 预测成功。
预测 1000 次 成功率= 0.001
y预测= 6YFt y实际= 6hFt 预测失败。
预测 1000 次 成功率= 0.001
y预测= NCC7 y实际= NCC7 预测成功。
预测 1000 次 成功率= 0.002
y预测= r304 y实际= r304 预测成功。
预测 1000 次 成功率= 0.003
y预测= mU93 y实际= mU93 预测成功。
预测 1000 次 成功率= 0.004
y预测= 8W78 y实际= 8W78 预测成功。
预测 1000 次 成功率= 0.005
y预测= JWHn y实际= JWHn 预测成功。
预测 1000 次 成功率= 0.006
y预测= Z614 y实际= Z614 预测成功。
预测 1000 次 成功率= 0.007
y预测= TLdS y实际= TLdS 预测成功。
预测 1000 次 成功率= 0.008
y预测= XKEN y实际= XKEN 预测成功。
预测 1000 次 成功率= 0.009
y预测= VXU8 y实际= VXU8 预测成功。
预测 1000 次 成功率= 0.01
y预测= 9HWQ y实际= 9HWQ 预测成功。
预测 1000 次 成功率= 0.011
y预测= 1eAh y实际= UeAh 预测失败。
预测 1000 次 成功率= 0.011
y预测= aS0S y实际= aS0S 预测成功。
预测 1000 次 成功率= 0.012
y预测= dk0b y实际= dk0b 预测成功。
预测 1000 次 成功率= 0.013
y预测= VntD y实际= VntD 预测成功。
预测 1000 次 成功率= 0.014
y预测= Andn y实际= Ardn 预测失败。
预测 1000 次 成功率= 0.014
y预测= P9fD y实际= P9fD 预测成功。
预测 1000 次 成功率= 0.015
y预测= e0rV y实际= e0rV 预测成功。
预测 1000 次 成功率= 0.016
y预测= Vk9X y实际= Vk9X 预测成功。
预测 1000 次 成功率= 0.017
y预测= NTM9 y实际= NTM9 预测成功。
预测 1000 次 成功率= 0.018
y预测= tS4W y实际=

y预测= dYJD y实际= dyJD 预测成功。
预测 1000 次 成功率= 0.145
y预测= GHRH y实际= GHRH 预测成功。
预测 1000 次 成功率= 0.146
y预测= Bm5n y实际= Bm5n 预测成功。
预测 1000 次 成功率= 0.147
y预测= RLmF y实际= CLmF 预测失败。
预测 1000 次 成功率= 0.147
y预测= 0rVP y实际= 0rVP 预测成功。
预测 1000 次 成功率= 0.148
y预测= 41MA y实际= 41MA 预测成功。
预测 1000 次 成功率= 0.149
y预测= 1LJY y实际= 1LJU 预测失败。
预测 1000 次 成功率= 0.149
y预测= 5ZN5 y实际= 5ZN5 预测成功。
预测 1000 次 成功率= 0.15
y预测= WHmk y实际= WHmk 预测成功。
预测 1000 次 成功率= 0.151
y预测= UmD7 y实际= UmD7 预测成功。
预测 1000 次 成功率= 0.152
y预测= UnT9 y实际= Unm9 预测失败。
预测 1000 次 成功率= 0.152
y预测= Vy8n y实际= Vy8n 预测成功。
预测 1000 次 成功率= 0.153
y预测= DkkF y实际= DkkM 预测失败。
预测 1000 次 成功率= 0.153
y预测= NMFS y实际= NMFS 预测成功。
预测 1000 次 成功率= 0.154
y预测= CmUW y实际= CmUW 预测成功。
预测 1000 次 成功率= 0.155
y预测= 8Bre y实际= 8Bre 预测成功。
预测 1000 次 成功率= 0.156
y预测= Q9P3 y实际= Q9P3 预测成功。
预测 1000 次 成功率= 0.157
y预测= TSDt y实际= TSDt 预测成功。
预测 1000 次 成功率= 0.158
y预测= Dt43 y实际= Dt43 预测成功。
预测 1000 次 成功率= 0.159
y预测= 92Fy y实际= 92Fy 预测成功。
预测 1000 次 成功率= 0.16
y预测= a3NW y实际= a3NW 预测成功。
预测 1000 次 成功率= 0.161
y预测= Dfnm y实际= 

y预测= b918 y实际= b918 预测成功。
预测 1000 次 成功率= 0.297
y预测= SF9F y实际= SF9F 预测成功。
预测 1000 次 成功率= 0.298
y预测= F8tf y实际= F8tf 预测成功。
预测 1000 次 成功率= 0.299
y预测= VTrH y实际= VTrH 预测成功。
预测 1000 次 成功率= 0.3
y预测= tTJ0 y实际= tTJ0 预测成功。
预测 1000 次 成功率= 0.301
y预测= KnDb y实际= KnQb 预测失败。
预测 1000 次 成功率= 0.301
y预测= LG06 y实际= LG06 预测成功。
预测 1000 次 成功率= 0.302
y预测= GrEZ y实际= GrEZ 预测成功。
预测 1000 次 成功率= 0.303
y预测= FB8M y实际= ZB8M 预测失败。
预测 1000 次 成功率= 0.303
y预测= rVde y实际= rVde 预测成功。
预测 1000 次 成功率= 0.304
y预测= TQkZ y实际= TQkZ 预测成功。
预测 1000 次 成功率= 0.305
y预测= SHkt y实际= SHkt 预测成功。
预测 1000 次 成功率= 0.306
y预测= KSrt y实际= KSrt 预测成功。
预测 1000 次 成功率= 0.307
y预测= yN4X y实际= yN47 预测失败。
预测 1000 次 成功率= 0.307
y预测= BAe3 y实际= BAe3 预测成功。
预测 1000 次 成功率= 0.308
y预测= SyDm y实际= SyDm 预测成功。
预测 1000 次 成功率= 0.309
y预测= eHGe y实际= 6HGe 预测失败。
预测 1000 次 成功率= 0.309
y预测= 90Nm y实际= 90ND 预测失败。
预测 1000 次 成功率= 0.309
y预测= 4CKE y实际= 4CKE 预测成功。
预测 1000 次 成功率= 0.31
y预测= dVNH y实际= dVNH 预测成功。
预测 1000 次 成功率= 0.311
y预测= Q7Am y实际= Q7Ym 预测失败。
预测 1000 次 成功率= 0.311
y预测= 2Anm y实际= 2

y预测= BUAZ y实际= BUAZ 预测成功。
预测 1000 次 成功率= 0.445
y预测= JnkS y实际= JnkS 预测成功。
预测 1000 次 成功率= 0.446
y预测= 3C7a y实际= 3C7a 预测成功。
预测 1000 次 成功率= 0.447
y预测= WW5D y实际= WW5D 预测成功。
预测 1000 次 成功率= 0.448
y预测= Wd4F y实际= Wd4F 预测成功。
预测 1000 次 成功率= 0.449
y预测= 1T77 y实际= 1T77 预测成功。
预测 1000 次 成功率= 0.45
y预测= NtJr y实际= NtJr 预测成功。
预测 1000 次 成功率= 0.451
y预测= 4Gyf y实际= 4Gyf 预测成功。
预测 1000 次 成功率= 0.452
y预测= P30k y实际= Pt0k 预测失败。
预测 1000 次 成功率= 0.452
y预测= 0rXZ y实际= 0rXZ 预测成功。
预测 1000 次 成功率= 0.453
y预测= 2WfZ y实际= 2WfZ 预测成功。
预测 1000 次 成功率= 0.454
y预测= CkQ6 y实际= CkQ6 预测成功。
预测 1000 次 成功率= 0.455
y预测= kXBX y实际= WXBX 预测失败。
预测 1000 次 成功率= 0.455
y预测= XDJt y实际= XDJt 预测成功。
预测 1000 次 成功率= 0.456
y预测= tK3G y实际= tK3G 预测成功。
预测 1000 次 成功率= 0.457
y预测= Zn0y y实际= Zn0y 预测成功。
预测 1000 次 成功率= 0.458
y预测= 3Rm1 y实际= 3Rm1 预测成功。
预测 1000 次 成功率= 0.459
y预测= P42U y实际= P42U 预测成功。
预测 1000 次 成功率= 0.46
y预测= r153 y实际= r153 预测成功。
预测 1000 次 成功率= 0.461
y预测= U2H6 y实际= U2H6 预测成功。
预测 1000 次 成功率= 0.462
y预测= ZNVa y实际= ZNVa 预测成功。
预测 1000 次 成功率= 0.463
y预测= J9ee y实际= 

y预测= LWJd y实际= LWJd 预测成功。
预测 1000 次 成功率= 0.591
y预测= mTP7 y实际= mTP7 预测成功。
预测 1000 次 成功率= 0.592
y预测= mm79 y实际= mm79 预测成功。
预测 1000 次 成功率= 0.593
y预测= JTef y实际= JUef 预测失败。
预测 1000 次 成功率= 0.593
y预测= 1QRK y实际= 1QRK 预测成功。
预测 1000 次 成功率= 0.594
y预测= nHJQ y实际= nHJQ 预测成功。
预测 1000 次 成功率= 0.595
y预测= D5K9 y实际= D5K9 预测成功。
预测 1000 次 成功率= 0.596
y预测= fRDF y实际= fRDF 预测成功。
预测 1000 次 成功率= 0.597
y预测= GHAW y实际= GHAW 预测成功。
预测 1000 次 成功率= 0.598
y预测= MhU9 y实际= MdU9 预测失败。
预测 1000 次 成功率= 0.598
y预测= YRhC y实际= YRhC 预测成功。
预测 1000 次 成功率= 0.599
y预测= aHCb y实际= aHCb 预测成功。
预测 1000 次 成功率= 0.6
y预测= e0eM y实际= e0eM 预测成功。
预测 1000 次 成功率= 0.601
y预测= V9CJ y实际= V9CJ 预测成功。
预测 1000 次 成功率= 0.602
y预测= NY46 y实际= NYR6 预测失败。
预测 1000 次 成功率= 0.602
y预测= 3W2r y实际= 3W2r 预测成功。
预测 1000 次 成功率= 0.603
y预测= 8M49 y实际= 8M49 预测成功。
预测 1000 次 成功率= 0.604
y预测= CmUW y实际= CmUW 预测成功。
预测 1000 次 成功率= 0.605
y预测= 3Je4 y实际= 3Je4 预测成功。
预测 1000 次 成功率= 0.606
y预测= NnCm y实际= NnCm 预测成功。
预测 1000 次 成功率= 0.607
y预测= BPWH y实际= BPFH 预测失败。
预测 1000 次 成功率= 0.607
y预测= hCbA y实际= 

y预测= SEAb y实际= SEAb 预测成功。
预测 1000 次 成功率= 0.735
y预测= HHS6 y实际= HHS6 预测成功。
预测 1000 次 成功率= 0.736
y预测= fMAA y实际= fMAA 预测成功。
预测 1000 次 成功率= 0.737
y预测= d1SD y实际= d1SD 预测成功。
预测 1000 次 成功率= 0.738
y预测= HV9k y实际= HV9k 预测成功。
预测 1000 次 成功率= 0.739
y预测= 3VQk y实际= 3VQk 预测成功。
预测 1000 次 成功率= 0.74
y预测= emTe y实际= emTe 预测成功。
预测 1000 次 成功率= 0.741
y预测= Da2d y实际= Da2d 预测成功。
预测 1000 次 成功率= 0.742
y预测= 8XT6 y实际= 8XT6 预测成功。
预测 1000 次 成功率= 0.743
y预测= fnRX y实际= fnRX 预测成功。
预测 1000 次 成功率= 0.744
y预测= aLXN y实际= aLXN 预测成功。
预测 1000 次 成功率= 0.745
y预测= C9bU y实际= C9bU 预测成功。
预测 1000 次 成功率= 0.746
y预测= 32QK y实际= 32QK 预测成功。
预测 1000 次 成功率= 0.747
y预测= krBr y实际= krBr 预测成功。
预测 1000 次 成功率= 0.748
y预测= BB68 y实际= BB68 预测成功。
预测 1000 次 成功率= 0.749
y预测= BTNa y实际= BVN5 预测失败。
预测 1000 次 成功率= 0.749
y预测= DKHA y实际= DKHA 预测成功。
预测 1000 次 成功率= 0.75
y预测= 4KL2 y实际= 4KL2 预测成功。
预测 1000 次 成功率= 0.751
y预测= QmT4 y实际= 4DT4 预测失败。
预测 1000 次 成功率= 0.751
y预测= YP3H y实际= YP3H 预测成功。
预测 1000 次 成功率= 0.752
y预测= V97k y实际= hA7k 预测失败。
预测 1000 次 成功率= 0.752
y预测= PD1C y实际= 

y预测= VmCT y实际= VmCT 预测成功。
预测 100 次 成功率= 0.01
y预测= YCVQ y实际= YCVQ 预测成功。
预测 100 次 成功率= 0.02
y预测= Z3kG y实际= Z3kG 预测成功。
预测 100 次 成功率= 0.03
y预测= NrYX y实际= NrYX 预测成功。
预测 100 次 成功率= 0.04
y预测= 9UfC y实际= 9UfC 预测成功。
预测 100 次 成功率= 0.05
y预测= ZdMN y实际= ZdMN 预测成功。
预测 100 次 成功率= 0.06
y预测= JdnA y实际= JdnA 预测成功。
预测 100 次 成功率= 0.07
y预测= 0hkV y实际= 0hkV 预测成功。
预测 100 次 成功率= 0.08
y预测= UNG9 y实际= UNG9 预测成功。
预测 100 次 成功率= 0.09
y预测= h229 y实际= h229 预测成功。
预测 100 次 成功率= 0.1
y预测= FG6N y实际= FG6N 预测成功。
预测 100 次 成功率= 0.11
y预测= Z76W y实际= Z76W 预测成功。
预测 100 次 成功率= 0.12
y预测= LTKG y实际= LTKG 预测成功。
预测 100 次 成功率= 0.13
y预测= 05KQ y实际= 05KQ 预测成功。
预测 100 次 成功率= 0.14
y预测= SMr5 y实际= SMr5 预测成功。
预测 100 次 成功率= 0.15
y预测= JmFf y实际= JmFf 预测成功。
预测 100 次 成功率= 0.16
y预测= m1r2 y实际= m1r2 预测成功。
预测 100 次 成功率= 0.17
y预测= XTFC y实际= XTFC 预测成功。
预测 100 次 成功率= 0.18
y预测= f6GU y实际= f6GU 预测成功。
预测 100 次 成功率= 0.19
y预测= M6WG y实际= M6WG 预测成功。
预测 100 次 成功率= 0.2
y预测= nF9Q y实际= nF9Q 预测成功。
预测 100 次 成功率= 0.21
y预测= 4Z8d y实际= 4Z8d 预测成功。
预测 100 次 成功率= 0.22
y预测= fCUH y实